In [1]:
%load_ext lab_black

from copy import deepcopy
from os import listdir
from os.path import isfile, join
from pathlib import Path
import matplotlib.pyplot as plt
import scipy.io
from PIL import Image
from PIL import ImageDraw
import json
from random import random
import numpy as np
import cv2
from tensorflow.keras.applications import imagenet_utils
from tensorflow.keras.applications.resnet50 import preprocess_input
import os
import errno, os, stat, shutil
from tqdm.notebook import tqdm
from enum import Enum

import tensorflow as tf
from keras.applications.vgg16 import VGG16
from keras.applications.vgg19 import VGG19

from keras.layers import *
from keras import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import categorical_crossentropy
from datetime import datetime
from focal_loss import BinaryFocalLoss

# tf.debugging.set_log_device_placement(True)
plt.rcParams["figure.figsize"] = [40, 20]
print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices("GPU")))

Num GPUs Available:  1


In [2]:
from datautils import *
from ArrayLoader import *
from CWDataset import *
from augmentation import *

In [3]:
original_path = Path("data/CW/original")
generated_path = Path("data/CW/generated")

In [4]:
real_shape = (544, 1280, 3)
nn_shape = (544 // 4, 1280 // 4, 3)
augmented_samples = 6

In [5]:
dataset = CWDataset(generated_path)

Train 1133
Test  149
Vali  148


In [6]:
def get_simple_conv_model(inp_shape):
    inp = Input(shape=inp_shape)
    headModel = Conv2D(32, (3, 3), activation="relu", input_shape=inp_shape)(inp)
    headModel = MaxPooling2D((2, 2))(headModel)
    headModel = Conv2D(64, (3, 3), activation="relu")(headModel)
    headModel = MaxPooling2D((2, 2))(headModel)
    headModel = Conv2D(86, (3, 3), activation="relu")(headModel)
    headModel = MaxPooling2D((2, 2))(headModel)
    headModel = Conv2D(86, (3, 3), activation="relu")(headModel)
    headModel = MaxPooling2D((2, 2))(headModel)

    headModel = Flatten(name="flatten")(headModel)
    headModel = Dense(256, activation="relu")(headModel)
    headModel = Dropout(0.2)(headModel)
    headModel = Dense(inp_shape[0] * inp_shape[1], activation="sigmoid")(headModel)
    headModel = Reshape((inp_shape[0], inp_shape[1]))(headModel)
    model = Model(inputs=inp, outputs=headModel)
    model.compile(
        loss=BinaryFocalLoss(gamma=3),
        optimizer=Adam(lr=0.001),
        metrics=["accuracy"],
    )
    return model

In [7]:
def train_conv_model():
    X, Y = prepare_dataset(*dataset.get_test(5000), nn_shape[:2])
    X_val, Y_val = prepare_dataset(*dataset.get_validation(5000), nn_shape[:2])

    simple_conv_model = get_simple_conv_model(nn_shape)
    print(simple_conv_model.summary())

    # simple_conv_model.load_weights("model/simple/08-12-2020 17-05-44.h5")

    my_callbacks = [
        tf.keras.callbacks.EarlyStopping(patience=30, monitor="val_loss"),
        #         tf.keras.callbacks.ModelCheckpoint(
        #             filepath="model/model.{epoch:02d}-{val_loss:.2f}.h5"
        #         ),
        tf.keras.callbacks.TensorBoard(log_dir="./logs"),
    ]

    with tf.device("/GPU:0"):
        simple_conv_model.fit(
            X, Y, epochs=100, validation_data=(X_val, Y_val), callbacks=my_callbacks
        )

    dt_string = datetime.now().strftime("model/simple/%d-%m-%Y %H-%M-%S.h5")
    print("saving as " + dt_string)
    simple_conv_model.save(dt_string)
    return simple_conv_model

In [8]:
simple_conv_model = train_conv_model()

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████| 148/148 [00:00<00:00, 371.30it/s]


Model: "functional_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 136, 320, 3)]     0         
_________________________________________________________________
conv2d (Conv2D)              (None, 134, 318, 32)      896       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 67, 159, 32)       0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 65, 157, 64)       18496     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 32, 78, 64)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 30, 76, 86)        49622     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 15, 38, 86)       

UnknownError:  Failed to get convolution algorithm. This is probably because cuDNN failed to initialize, so try looking to see if a warning log message was printed above.
	 [[node functional_1/conv2d/Relu (defined at <ipython-input-7-1edc0db357d7>:19) ]] [Op:__inference_train_function_1201]

Function call stack:
train_function


In [ ]:
# show_on_data(*dataset.get_train(5), 5, simple_conv_model, threshold = 0)

In [ ]:
show_on_data(*dataset.get_test(5), 5, simple_conv_model, nn_shape[:2])

In [ ]:
def show_labeled(arrays, labels, model, threshold=0.7):
    X, Y = prepare_dataset(arrays, labels, nn_shape[:2])
    pred = model.predict(X)
    pred_labels = [probs_to_labels(pred[i], threshold, 4) for i in range(len(pred))]

    plot_images(arrays, 5000, labels, pred_labels)

In [ ]:
show_labeled(*dataset.get_train(10), simple_conv_model, 0.5)

In [ ]:
show_labeled(*dataset.get_test(10), simple_conv_model, 0.5)

In [ ]:
def calculate_iou_levels_simpl_conv(X, Y, threshold=0.3):
    X_in = prepare_dataset(X, Y, nn_shape[:2])
    predictions = simple_conv_model.predict(X_in)
    pred_labels = [
        probs_to_labels(predictions[i], threshold, 4) for i in range(len(predictions))
    ]
    for iou in range(1, 11):
        print(iou / 10.0, accuracy(Y, pred_labels, iou / 10.0))

In [ ]:
print("train")
calculate_iou_levels_simpl_conv(*dataset.get_train(5000))
print("test")
calculate_iou_levels_simpl_conv(*dataset.get_test(5000))
print("validation")
calculate_iou_levels_simpl_conv(*dataset.get_validation(5000))

In [ ]:
%reset -f array